# Libs

In [5]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn.metrics.pairwise import euclidean_distances
from tqdm import tqdm
import statistics
import math
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from scipy.spatial import ConvexHull
from matplotlib.patches import Circle

import geopandas as gpd
from shapely.geometry import Point

from scipy.linalg import solve
from itertools import product
from sklearn.metrics import mean_squared_error as MSE

pd.options.display.precision = 3
pd.options.display.float_format = lambda x: '%.5f' % x
pd.options.display.max_columns = 15
pd.options.display.max_rows = 6

# Data uploading

In [6]:
df_bal8_v4 = pd.read_csv('C:\jupyter\SPP\inputoutput\general_logs\df_bal8_azr_v4.csv')
df_bal8_v4.columns = df_bal8_v4.columns.str.lower()
df_bal8_v4.loc[df_bal8_v4.formation=='Balakhany VIII sand','formation'] = '1_bal8_sand'
df_bal8_v4.loc[df_bal8_v4.formation=='Balakhany VIII 25','formation'] = '2_bal8_25'
df_bal8_v4.loc[df_bal8_v4.formation=='Balakhany VIII 20','formation'] = '3_bal8_20'
df_bal8_v4.loc[df_bal8_v4.formation=='Balakhany VIII 15','formation'] = '4_bal8_15'
df_bal8_v4.loc[df_bal8_v4.formation=='Balakhany VIII 10','formation'] = '5_bal8_10'
df_bal8_v4.loc[df_bal8_v4.formation=='Balakhany VIII 5','formation'] = '6_bal8_5'
well_phit_flag8 = df_bal8_v4[df_bal8_v4.phit_flag==1].groupby('well')['phit_flag'].apply(lambda x: x.iloc[0]).reset_index().well.unique()
df_bal8_v4_flag = df_bal8_v4[df_bal8_v4.well.isin(well_phit_flag8)]

df_bal10_v4 = pd.read_csv('C:\jupyter\SPP\inputoutput\general_logs\df_bal10_vshclp2_v4.csv')
df_bal10_v4.columns = df_bal10_v4.columns.str.lower()
df_bal10_v4.loc[df_bal10_v4.formation=='Balakhany X sand','formation'] = '1_bal10_sand'
df_bal10_v4.loc[df_bal10_v4.formation=='Balakhany X 50','formation'] = '2_bal10_40'
df_bal10_v4.loc[df_bal10_v4.formation=='Balakhany X 40','formation'] = '2_bal10_40'
df_bal10_v4.loc[df_bal10_v4.formation=='Balakhany X 20','formation'] = '3_bal10_20'
well_phit_flag10 = df_bal10_v4[df_bal10_v4.phit_flag==1].groupby('well')['phit_flag'].apply(lambda x: x.iloc[0]).reset_index().well.unique()
df_bal10_v4_flag = df_bal10_v4[df_bal10_v4.well.isin(well_phit_flag10)]

ntd_top_phi_bot8_bp_v4 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\layers\ntd_top_phi_bot8_bp_v4.csv').drop('Unnamed: 0', axis=1)
ntd_top_phi_bot8_bp_v4.columns = ntd_top_phi_bot8_bp_v4.columns.str.lower()
ntd_top_phi_bot10_bp_v4 = pd.read_csv(r'C:\jupyter\SPP\inputoutput\layers\ntd_top_phi_bot10_bp_v4.csv').drop('Unnamed: 0', axis=1)
ntd_top_phi_bot10_bp_v4.columns = ntd_top_phi_bot10_bp_v4.columns.str.lower()

In [8]:
def well_dist_calc(dataset, fm):
    data = dataset.groupby('well')[['xmean', 'ymean']].first().reset_index().dropna()
    row_name = data.well.reset_index().drop(['index'], axis=1)
    distance_fm = pd.DataFrame(euclidean_distances(data[['xmean', 'ymean']]), columns=list(data.well))
    distance_fm_well = distance_fm.join(row_name).set_index('well')
    distance_fm_well = distance_fm_well.reset_index()
    dist_melt = distance_fm_well.melt(id_vars='well', 
                                var_name='well2', 
                                value_name='dist').rename(columns={'well':'well_offset', 'well2':'well'})
    dist_melt = dist_melt[['well', 'well_offset', 'dist']]
    dist_melt = dist_melt[dist_melt.dist != 0].sort_values(by=['well','dist'])
    dist_melt['FORMATION_up'] = fm
    return dist_melt

dist_bal8 = well_dist_calc(df_bal8_v4_flag, 'Balakhany VIII')
dist_bal10 = well_dist_calc(df_bal10_v4_flag, 'Balakhany X')

# Well pairs

In [9]:
# df_lst = []
# for wellname in dist_bal8.well.unique():
#     data = dist_bal8[dist_bal8.well == wellname].iloc[0:1]
#     df_lst.append(data)
# well_pairs = pd.concat(df_lst).reset_index(drop=True).drop_duplicates(subset=['dist'])
# well_pairs_v2 = well_pairs[well_pairs.dist < 500] # 235m is the max distance between wells selected by elbow plot with distance 500m
# # well_pairs.hist(column='dist', bins=50)
# well_pairs_v2.sort_values(by='dist', ascending=True)

In [10]:
# fig = plt.figure(figsize=(20, 5))
# sns.lineplot(data=well_pairs_v2.sort_values(by='dist', ascending=False), x='well', y='dist')
# plt.xticks(rotation=90)
# plt.grid()

In [11]:
# df_lst = []
# for wellname in dist_bal8.well.unique():
#     data = dist_bal8[dist_bal8.well == wellname].iloc[0:1]
#     df_lst.append(data)
# well_pairs = pd.concat(df_lst).reset_index(drop=True).drop_duplicates(subset=['dist'])
# well_pairs_v3 = well_pairs[well_pairs.dist < 230] # 235m is the max distance between wells selected by elbow plot with distance 500m
# # well_pairs.hist(column='dist', bins=50)
# well_pairs_v3 = well_pairs_v3.sort_values(by='dist', ascending=True).reset_index(drop=True)
# well_pairs_v3

In [12]:
# def well_offset_comparison_dashboard_pairs(dataset_wells, dataset_layers, dist_df,  well_target, offset_qty, fm_name, print_flag):
#     offset_well_list = dist_df[dist_df.well == well_target].iloc[:offset_qty]['well_offset'].values.tolist()
#     # offset_well_list = dist_df[dist_df.well == well_target].iloc[:offset_qty]
#     well_list = [well_target] + offset_well_list
#     data_logs = dataset_wells[(dataset_wells.well.isin(well_list)) & (dataset_wells.phit != 0)]
#     data_layers = dataset_layers[   (dataset_layers.well.isin(well_list)) & 
#                                     (dataset_layers.htst > 1)]
#     khtst_logs = data_logs.groupby(['well','formation'])[['khtst']].apply(lambda x: x.iloc[0] - x.iloc[-1]).reset_index()

#     def khtst_layer_calculation(data_logs):
#         data = data_logs[data_logs.net == 1]
#         df_lst = []
#         for wellname in data.well.unique():
#             well_data = data_logs[data_logs.well == wellname]
#             well_data['tst_index_rev'] = [i for i in range(len(well_data['tst']))[::-1]]
#             df_lst.append(well_data)
#         data_logs_khtst = pd.concat(df_lst)
#         return data_logs_khtst
#     data_logs_khtst = khtst_layer_calculation(data_logs)

#     def well_dist_title(dist_df):
#         offset_well_list = dist_df[dist_df.well == well_target].iloc[:offset_qty]
#         well = offset_well_list['well'].iloc[0]
#         well1 = offset_well_list.iloc[0,1]
#         dist1 = offset_well_list.iloc[0,2].round(0).astype(int)
#         # well2 = offset_well_list.iloc[1,1]
#         # dist2 = offset_well_list.iloc[1,2].round(0).astype(int)
#         # well3 = offset_well_list.iloc[2,1]
#         # dist3 = offset_well_list.iloc[2,2].round(0).astype(int)
#         return f"target well {well} : offsets {well1} - {dist1}m orange;"

#     fig = plt.figure(figsize=(22, 10))
#     gs = gridspec.GridSpec(2, 4, figure=fig)
#     ax1 = fig.add_subplot(gs[0, 0])
#     ax2 = fig.add_subplot(gs[0, 1])
#     ax3 = fig.add_subplot(gs[0, 2])
#     ax4 = fig.add_subplot(gs[0, 3])
#     ax5 = fig.add_subplot(gs[1, :3])

#     custom_palette = {well_target: 'red', offset_well_list[0]: 'orange'}
#     sns.histplot(data=data_logs, x='phit', hue='well', bins=50, kde=True, ax=ax1, palette=custom_palette)
#     sns.scatterplot(data=data_layers, x='htst', y='perm_avg', hue='well', s=75, ax=ax2, alpha=0.5, ec='black', palette=custom_palette)
#     sns.lineplot(data=data_logs_khtst, x='tst_index_rev', y='khtst', hue='well', ax=ax3, palette=custom_palette)
#     sns.barplot(data = khtst_logs, x='formation', y='khtst', hue='well', ax=ax4, palette=custom_palette)
#     ax1.set_yticklabels(ax1.get_yticklabels(), rotation=90, va='center')
#     ax2.set_yscale('log')
#     ax2.grid(True, which='both', linestyle='--', linewidth=0.5)
#     ax2.set_yticklabels(ax2.get_yticklabels(), rotation=90, va='center')
#     ax3.grid(True, which='both', linestyle='--', linewidth=0.5)
#     ax3.set_yticklabels(ax3.get_yticklabels(), rotation=90, va='center')

#     x = np.arange(len(khtst_logs.formation.unique()))
#     fms = khtst_logs.formation.unique()
#     ax4.set_xticks(x, fms, rotation=45, fontsize=6)
#     ax4.set_yticklabels(ax4.get_yticklabels(), rotation=90, va='center')

#     offset_well_list = dist_df[dist_df.well == well_target].iloc[:offset_qty]['well_offset'].values.tolist()
#     x = dataset_wells[dataset_wells.phit_flag == 1]['xmean']
#     y = dataset_wells[dataset_wells.phit_flag == 1]['ymean']
#     x_target = dataset_wells[dataset_wells.well == well_target]['xmean'].iloc[0]
#     y_target = dataset_wells[dataset_wells.well == well_target]['ymean'].iloc[0]
#     x_well1 = dataset_wells[dataset_wells.well == offset_well_list[0]]['xmean'].iloc[0]
#     y_well1 = dataset_wells[dataset_wells.well == offset_well_list[0]]['ymean'].iloc[0]
#     # x_well2 = dataset_wells[dataset_wells.well == offset_well_list[1]]['xmean'].iloc[0]
#     # y_well2 = dataset_wells[dataset_wells.well == offset_well_list[1]]['ymean'].iloc[0]
#     # x_well3 = dataset_wells[dataset_wells.well == offset_well_list[2]]['xmean'].iloc[0]
#     # y_well3 = dataset_wells[dataset_wells.well == offset_well_list[2]]['ymean'].iloc[0]
#     ax5.scatter(x, y, color='gray', s=10)
#     ax5.scatter(x_target, y_target, color='red', s=50, ec='black')
#     ax5.scatter(x_well1, y_well1, color='orange')
#     # ax5.scatter(x_well2, y_well2, color='green')
#     # ax5.scatter(x_well3, y_well3, color='#0797eb')

#     plt.suptitle(well_dist_title(dist_df), fontsize=16, y=0.92, x=0.32)
#     if print_flag == 'print':
#         plt.savefig(f'C:/jupyter/SPP/plots/offset_dashboard/{fm_name}_{well_target}_offset_dashboard.png');

# df_bal8_v4_flag = df_bal8_v4_flag[~df_bal8_v4_flag.well.isin(['E31Z', 'D01Z'])]
# dist_bal8 = well_dist_calc(df_bal8_v4_flag, 'Balakhany VIII').round(0)
# for wellname in well_pairs_v3.well:
#     try:
#         well_offset_comparison_dashboard_pairs(df_bal8_v4_flag, ntd_top_phi_bot8_bp_v4, well_pairs_v3, wellname, 1, 'bal8','dontprint')
#     except:
#         print(f"error in {wellname}")

# Maps input

In [13]:
ntd_top_phi_bot8_bp_v4['htst*phit_avg'] = ntd_top_phi_bot8_bp_v4['htst'] * ntd_top_phi_bot8_bp_v4['phit_avg']
ntd8 = ntd_top_phi_bot8_bp_v4.groupby('well')[['phit_avg','htst', 'htst*phit_avg']].agg({'phit_avg':'mean','htst':'sum', 'htst*phit_avg':'sum'}).reset_index()
xy = df_bal8_v4_flag.groupby('well')[['xmean','ymean']].first().reset_index()
ntd8 = ntd8.merge(xy, on='well').round({'xmean':0, 'ymean':0})
ntd8 = ntd8[~ntd8.well.isin(['B01ST1', 'D01', 'C14', 'C01A', 'B06', 'C13Z', 'C06', 'D01Z','C07'])] # remove wells pair wells
ntd8['phit_w_avg'] = ntd8['htst*phit_avg'] / ntd8['htst']
ntd8 = ntd8[['well', 'phit_avg', 'htst', 'phit_w_avg', 'xmean', 'ymean']]

x_min8 = ntd8.xmean.min()
x_max8 = ntd8.xmean.max()
y_min8 = ntd8.ymean.min()
y_max8 = ntd8.ymean.max()
margin8 = 250
X_8, Y_8 = np.arange(x_min8-margin8, x_max8+margin8, 50), np.arange(y_min8-margin8, y_max8+margin8, 30)

In [14]:
ntd_top_phi_bot10_bp_v4['htst*phit_avg'] = ntd_top_phi_bot10_bp_v4['htst'] * ntd_top_phi_bot10_bp_v4['phit_avg']
ntd10 = ntd_top_phi_bot10_bp_v4.groupby('well')[['phit_avg','htst', 'htst*phit_avg']].agg({'phit_avg':'mean','htst':'sum', 'htst*phit_avg':'sum'}).reset_index()
xy = df_bal10_v4_flag.groupby('well')[['xmean','ymean']].first().reset_index()
ntd10 = ntd10.merge(xy, on='well').round({'xmean':0, 'ymean':0})
ntd10 = ntd10[~ntd10.well.isin(['A13Z', 'H01Y', 'A01X', 'A12X', 'D01', 'E31', 'G01Y', 
                                'A09Y','E01X', 'A07Y', 'A01W', 'C27Y', 'C03ST1', 'H01', 'D07'])] # remove wells pair wells
ntd10['phit_w_avg'] = ntd10['htst*phit_avg'] / ntd10['htst']
ntd10 = ntd10[['well', 'phit_avg', 'htst', 'phit_w_avg', 'xmean', 'ymean']]

x_min10 = ntd10.xmean.min()
x_max10 = ntd10.xmean.max()
y_min10 = ntd10.ymean.min()
y_max10 = ntd10.ymean.max()
margin10 = 250
X_10, Y_10 = np.arange(x_min10-margin10, x_max10+margin10, 50), np.arange(y_min10-margin10, y_max10+margin10, 30)

# NN

In [ ]:
sns.scatterplot(data=ntd8, x='phit_avg', y='phit_w_avg', hue='htst', palette='viridis')
sns.lineplot(x=[0.15,0.27], y=[0.15,0.27], color='red', ls='--')
plt.grid()

In [16]:
# latitude - north-south
# longitude - east-west

def Euclidean(x1,x2,y1,y2):
  return ((x1-x2)**2+(y1-y2)**2)**0.5

def NN(data, Y, X, var):
  array = np.empty((Y.shape[0], X.shape[0]))
  for i, y in enumerate(Y):
    for j, x in enumerate(X):
      idx = data.apply(lambda row: Euclidean(row.xmean, x, row.ymean, y), axis = 1).argmin()
      array[i,j] = data.iloc[idx][var]
  return array

In [17]:
# fig, ax = plt.subplots(figsize=(15,7))
# xy = ntd8[['xmean', 'ymean']].values
# hull = ConvexHull(xy)
# hull_points = xy[hull.vertices]

# phit_nn = NN(ntd8, Y, X, 'phit_avg')
# nn_phit_map = pd.DataFrame(phit_nn, index=Y, columns=X)
# nn_phit_map_v2 = nn_phit_map.reset_index().melt(id_vars='index', var_name='Category', value_name='Value')
# nn_phit_map_v2 = nn_phit_map_v2.rename(columns={'index':'y', 'Category':'x', 'Value':'phit_avg'})

# sc = plt.scatter(nn_phit_map_v2.x, nn_phit_map_v2.y, s=2, c=nn_phit_map_v2.phit_avg, cmap='viridis', alpha=0.5)
# plt.colorbar(sc)
# plt.scatter(ntd8.xmean, ntd8.ymean, s=50, c=ntd8.phit_avg, marker='o', ec='black', cmap='viridis', lw=0.5)
# plt.title('phit_avg')

# for simplex in hull.simplices:
#     plt.plot(xy[simplex, 0], xy[simplex, 1], 'k-');

In [18]:
# phit_nn = NN(ntd8, Y, X, 'phit_avg')
# nn_phit_map = pd.DataFrame(phit_nn, index=Y, columns=X)
# nn_phit_map_v2 = nn_phit_map.reset_index().melt(id_vars='index', var_name='Category', value_name='Value')
# nn_phit_map_v2 = nn_phit_map_v2.rename(columns={'index':'y', 'Category':'x', 'Value':'phit_avg'})
# nn_phit_map_v2.to_csv('C:/jupyter/SPP/inputoutput/nn_phit_map_v2.csv', index=False)

In [19]:
# w_phit_nn8 = NN(ntd8, Y_8, X_8, 'phit_w_avg')
# nn_w_phit_map8 = pd.DataFrame(w_phit_nn8, index=Y_8, columns=X_8)
# nn_w_phit_map8_v2 = nn_w_phit_map8.reset_index().melt(id_vars='index', var_name='Category', value_name='Value')
# nn_w_phit_map8_v2 = nn_w_phit_map8_v2.rename(columns={'index':'y', 'Category':'x', 'Value':'phit_avg'})
# nn_w_phit_map8_v2.to_csv('C:/jupyter/SPP/inputoutput/nn_w_phit_map8_v2.csv', index=False)

In [20]:
# w_phit_nn10 = NN(ntd10, Y_10, X_10, 'phit_w_avg')
# nn_w_phit_map10 = pd.DataFrame(w_phit_nn10, index=Y_10, columns=X_10)
# nn_w_phit_map10_v2 = nn_w_phit_map10.reset_index().melt(id_vars='index', var_name='Category', value_name='Value')
# nn_w_phit_map10_v2 = nn_w_phit_map10_v2.rename(columns={'index':'y', 'Category':'x', 'Value':'phit_avg'})
# nn_w_phit_map10_v2.to_csv('C:/jupyter/SPP/inputoutput/nn_w_phit_map10_v2.csv', index=False)

In [21]:
def intersection_maps(map, wells, buffer):
    geometry_map = [Point(xy) for xy in zip(map['x'], map['y'])]
    gdf_map = gpd.GeoDataFrame(map, geometry=geometry_map)

    geometry_points = [Point(xy) for xy in zip(wells['xmean'], wells['ymean'])]
    gdf_points = gpd.GeoDataFrame(wells, geometry=geometry_points)
    convex_hull = gdf_points.unary_union.convex_hull.buffer(buffer)
    intersection = gdf_map[gdf_map.intersects(convex_hull)]
    return intersection

nn_w_phit_map8_v2 = pd.read_csv('C:/jupyter/SPP/inputoutput/nn_w_phit_map8_v2.csv')
nn_w_phit_map10_v2 = pd.read_csv('C:/jupyter/SPP/inputoutput/nn_w_phit_map10_v2.csv')

intersection_phit8 = intersection_maps(nn_w_phit_map8_v2, ntd8, 1500)
intersection_phit10 = intersection_maps(nn_w_phit_map10_v2, ntd10, 1500)

In [ ]:
anomaly_gas_wells = ntd8[ntd8.well.isin(['B01Y','B13ST2','B14Z','B19','B26'])]
fig, ax = plt.subplots(figsize=(15,7))
sc = plt.scatter(intersection_phit8.x, intersection_phit8.y, s=2, c=intersection_phit8.phit_avg, cmap='viridis', alpha=0.5)
plt.colorbar(sc)
plt.scatter(ntd8.xmean, ntd8.ymean, s=50, c=ntd8.phit_w_avg, marker='o', ec='black', cmap='viridis', lw=0.5)
plt.title('phit_w_avg bal8')
for idx, txt in enumerate(ntd8.well):
    ax.annotate(txt, (ntd8.xmean.iloc[idx], ntd8.ymean.iloc[idx]), fontsize=6);
for (xi, yi) in anomaly_gas_wells[['xmean', 'ymean']].values:
    circle = Circle((xi, yi), 200, color='r', fill=False)
    ax.add_patch(circle);

In [ ]:
fig, ax = plt.subplots(figsize=(18,7))
sc = plt.scatter(intersection_phit10.x, intersection_phit10.y, s=2, c=intersection_phit10.phit_avg, cmap='viridis', alpha=0.5)
plt.colorbar(sc)
plt.scatter(ntd10.xmean, ntd10.ymean, s=50, c=ntd10.phit_w_avg, marker='o', ec='black', cmap='viridis', lw=0.5)
plt.title('phit_w_avg bal10')
for idx, txt in enumerate(ntd10.well):
    ax.annotate(txt, (ntd10.xmean.iloc[idx], ntd10.ymean.iloc[idx]), fontsize=6);

In [24]:
# htst_nn8 = NN(ntd8, Y_8, X_8, 'htst')
# nn_htst_map8 = pd.DataFrame(htst_nn8, index=Y_8, columns=X_8)
# nn_htst_map8_v2 = nn_htst_map8.reset_index().melt(id_vars='index', var_name='Category', value_name='Value')
# nn_htst_map8_v2 = nn_htst_map8_v2.rename(columns={'index':'y', 'Category':'x', 'Value':'htst'})
# nn_htst_map8_v2.to_csv('C:/jupyter/SPP/inputoutput/nn_htst_map8_v2.csv', index=False)

In [25]:
# htst_nn10 = NN(ntd10, Y_10, X_10, 'htst')
# nn_htst_map10 = pd.DataFrame(htst_nn10, index=Y_10, columns=X_10)
# nn_htst_map10_v2 = nn_htst_map10.reset_index().melt(id_vars='index', var_name='Category', value_name='Value')
# nn_htst_map10_v2 = nn_htst_map10_v2.rename(columns={'index':'y', 'Category':'x', 'Value':'htst'})
# nn_htst_map10_v2.to_csv('C:/jupyter/SPP/inputoutput/nn_htst_map10_v2.csv', index=False)

In [ ]:
nn_htst_map8_v2 = pd.read_csv('C:/jupyter/SPP/inputoutput/nn_htst_map8_v2.csv')
intersection_htst8 = intersection_maps(nn_htst_map8_v2, ntd8, 1500)
fig, ax = plt.subplots(figsize=(15,7))
sc = plt.scatter(intersection_htst8.x, intersection_htst8.y, s=2, c=intersection_htst8.htst, cmap='cividis', alpha=0.5)
plt.colorbar(sc)
plt.scatter(ntd8.xmean, ntd8.ymean, s=50, c=ntd8.htst, marker='o', ec='black', cmap='cividis', lw=0.5)
plt.title('htst')
for idx, txt in enumerate(ntd8.well):
    ax.annotate(txt, (ntd8.xmean.iloc[idx], ntd8.ymean.iloc[idx]), fontsize=6);

In [ ]:
nn_htst_map10_v2 = pd.read_csv('C:/jupyter/SPP/inputoutput/nn_htst_map10_v2.csv')
intersection_htst10 = intersection_maps(nn_htst_map10_v2, ntd10, 1500)
fig, ax = plt.subplots(figsize=(18,7))
sc = plt.scatter(intersection_htst10.x, intersection_htst10.y, s=2, c=intersection_htst10.htst, cmap='cividis', alpha=0.5)
plt.colorbar(sc)
plt.scatter(ntd10.xmean, ntd10.ymean, s=50, c=ntd10.htst, marker='o', ec='black', cmap='cividis', lw=0.5)
plt.title('htst')
for idx, txt in enumerate(ntd10.well):
    ax.annotate(txt, (ntd10.xmean.iloc[idx], ntd10.ymean.iloc[idx]), fontsize=6);

# IWD

In [28]:
# # latitude - north-south
# # longitude - east-west

# def IDW(data, Y, X, var, betta=10):
#   array = np.empty((Y.shape[0], X.shape[0]))
#   for i, y in enumerate(Y):
#     for j, x in enumerate(X):
#       weights = data.apply(lambda row: Euclidean(row.xmean, x, row.ymean, y)**(-betta), axis = 1)
#       z = sum(weights*data[var])/weights.sum()
#       array[i,j] = z
#   return array

# phit_idw = IDW(ntd8, Y, X, 'phit_avg',10)
# iwd_phit_map = pd.DataFrame(phit_idw, index=Y, columns=X)
# iwd_phit_map_v2 = iwd_phit_map.reset_index().melt(id_vars='index', var_name='Category', value_name='Value')
# iwd_phit_map_v2 = iwd_phit_map_v2.rename(columns={'index':'y', 'Category':'x', 'Value':'phit_avg'})
# fig, ax = plt.subplots(figsize=(15,7))
# sc = plt.scatter(iwd_phit_map_v2.x, iwd_phit_map_v2.y, s=2, c=iwd_phit_map_v2.phit_avg, cmap='viridis', alpha=0.5)
# plt.colorbar(sc)
# plt.scatter(ntd8.xmean, ntd8.ymean, s=50, c=ntd8.phit_avg, marker='o', ec='black', cmap='viridis', lw=0.5);

In [29]:
# htst_idw = IDW(ntd8, Y, X, 'htst', 10)
# htst_idw_map = pd.DataFrame(htst_idw, index=Y, columns=X)
# htst_idw_map_v2 = htst_idw_map.reset_index().melt(id_vars='index', var_name='Category', value_name='Value')
# htst_idw_map_v2 = htst_idw_map_v2.rename(columns={'index':'y', 'Category':'x', 'Value':'phit_avg'})
# fig, ax = plt.subplots(figsize=(15,7))
# sc = plt.scatter(htst_idw_map_v2.x, htst_idw_map_v2.y, s=2, c=htst_idw_map_v2.phit_avg, cmap='cividis', alpha=0.5)
# plt.colorbar(sc)
# plt.scatter(ntd8.xmean, ntd8.ymean, s=50, c=ntd8.phit_avg, marker='o', ec='black', cmap='cividis', lw=0.5);